# Generate LPCs From Audio Files

In [15]:
import librosa
import numpy as np

## Declare Audio Processing Constants

In [16]:
# Number of LPC Coefficients to Model 
NUM_LPC = 14
# Sample Rate is 44.1 kHz
RECORDED_SAMPLE_RATE = 44100

# Desired Sample Rate for LPC Processing is 14 kHz
DESIRED_SAMPLE_RATE = NUM_LPC * 1000

## Helper Functions

### Load Audio and Preprocess Audio
1. Downsampling 44kHz to 14kHz (1kHz per LPC Value)
2. Use a High Frequency Heuristic

In [17]:
# ./wav_data/vowels_Maria/aa_maria.wav
def process_audio(path_to_audio_file: str, HIGH_FREQUENCY_HEURISTIC=True) -> np.ndarray:
    """
    Processes an audio file by loading it into a numpy array
    and changing the sample rate to NUM_LPC * 1000

    Additionally, there is a heuristic for adding back frequencies
    using a simple `diff` of consecutive elements which can
    be disabled using the HIGH_FREQUENCY_HEURISTIC flag.

    Parameters
    ----------
    path_to_audio_file : str
        Path to the audio input file

    HIGH_FREQUENCY_HEURISTIC : bool
        Whether or not to use transform the audio file into a difference
        of consecutive samples. a[i] = a[i+1] - a[i] 

    Returns
    --------
    audio_array : np.ndarray
        processed audio file as a numpy array
    """

    # Step 1) Load in Raw Audio
    audio_file, _ = librosa.load(
        path=path_to_audio_file,
        sr=RECORDED_SAMPLE_RATE
    )

    # Step 2) Change sample rate
    down_sampled_audio_file = librosa.resample(
        audio_file,
        orig_sr=RECORDED_SAMPLE_RATE,
        target_sr=DESIRED_SAMPLE_RATE
    )

    # Step 3) Rentroduce high frequencies if needed
    if (HIGH_FREQUENCY_HEURISTIC):
        down_sampled_audio_file = np.diff(down_sampled_audio_file)

    return down_sampled_audio_file

## Cohen's D
- Calculate the effect size between 2 given LPC Coefficients

#### Cohen's D Intuition
- Cohen's D = measure of the `average difference` between 2 datasets and how prominent that variation is
- The lower the `normalized_variance` the more impactful the `mean_difference` is

#### Cohen's D Calculation
- Cohen's D = `mean_difference` / `normalized_variance`
- let `mean_difference` = avg(`dataset1's LPC Coefficient X`) - avg(`dataset2's LPC Coefficient X`)
- let `normalized_variance` = sqrt(`variance`) / 2
- let `variance` = std(`dataset1's LPC Coefficient X`)^2 - std(`dataset2's LPC Coefficient X`)^2

#### Cohen's D Ranges

| Effect Size | `Cohen's D` Value |
| ----------- | ----------------- |
| Small       | <= 0.2            |
| Medium      | <= 0.5            |
| Large       | >= 0.8            |

In [36]:
def calculate_cohens_d(lpc_x_dataset1: np.array, lpc_x_dataset2: np.array) -> tuple[np.float32, str]:
    """
    Compare the effect size of dataset1's and dataset2's LPC_X where
    X is an arbitrary LPC Coefficient

    Parameters
    ----------
    lpc_x_dataset1 : np.array
        The Xth LPC coefficients for every sample in dataset 1
            ex: LPC 12 for all `front vowels`
    
    lpc_x_dataset2 : np.array
        The Xth LPC coefficients for every sample in dataset 2
            ex: LPC 12 for all `front vowels`
    
    Note: Both datasets should be have the same LPC index

    Returns
    --------
    cohens_d : np.float64
        Cohen's D or Effect Size of LPC Index X with respect to dataset 1 and dataset 2
            ex: How likely the difference between LPC X in dataset 1 and dataset 2 
                is to be noticeable in an experiment/real life scenario

    label: str
        Returns whether the effect size is Small, Medium, or Large
    """

    assert len(lpc_x_dataset1) > 1, "lpc_x_dataset1 must have more than 1 number" 
    assert len(lpc_x_dataset1) > 1, "lpc_x_dataset2 must have more than 1 number" 

    # Step 1) Numerator = Mean Difference
    mean_difference = np.mean(lpc_x_dataset1) - np.mean(lpc_x_dataset2)
    
    # Step 2) Get the total variance of both datasets
    total_variance = np.std(lpc_x_dataset1)**2 + np.std(lpc_x_dataset2)**2

    # Step 3) Normalize the Variance
    normalized_variance = np.sqrt(total_variance) / 2

    # Step 4) Calculate effect size
    cohens_d = mean_difference / normalized_variance

    # Step 5) Get the label according to Cohen's D
    label = ""
    if(cohens_d <= 2.0):
        label = "Small"
    elif(cohens_d <= 5.0):
        label = "Medium"
    else:
        label = "Large"

    return cohens_d, label

cohens_d, label = calculate_cohens_d([10,12],[2,3])
print(cohens_d, label)

15.205262246998569 Large


In [19]:
lpc_vals = librosa.lpc(process_audio("./wav_data/vowels_Maria/aa_maria.wav"), order=NUM_LPC)

In [20]:
print(lpc_vals)
len(lpc_vals)

[ 1.         -1.2961028   1.0199971  -1.090959    1.2519653  -1.1698215
  1.392889   -1.2056091   1.2673672  -1.1874472   0.77811205 -0.42215684
  0.37347627 -0.25114942  0.15072984]


15